In [ ]:
import pandas as pd
import numpy as np
import json

from tqdm.auto import tqdm
import openai

from src.utils import call_gpt3_api

In [ ]:
# Set up OpenAI API credentials for GPT-3.5
openai_key = open("../keys/openai_key.txt", "r")
openai.api_key = openai_key.readlines()[0]

organization_id = open("../keys/rajpurkarlab_org_id.txt", "r")
openai.organization = organization_id.readlines()[0]

In [ ]:
def convert_to_vignette_prompt_gpt3(pat_dialogues):
    system_prompt = "Convert the vignette into 3rd person - .\n"
    system_prompt += pat_dialogues
    system_prompt += "\n\nFor example: \nOriginal Vignette - 'I have noticed that the skin around my hands," 
    system_prompt += "forearms, and face has been getting thicker over the past several months. "
    system_prompt += "I am 47 years old. I am a woman. "
    system_prompt += "I didn't mention any joint pain or stiffness in my described symptoms. "
    system_prompt += "I have no family history of autoimmune disease. "
    system_prompt += "I have not mentioned having any red or purple patches on my skin. "
    system_prompt += "I haven't mentioned any rapid weight gain in my face, arms, or legs. "
    system_prompt += "Yes, I am experiencing thickening of my skin around my hands, forearms, and face. "
    system_prompt += "I am not taking any medications.' "
    system_prompt += "\n\nConverted Vignette - 'A 47-year-old woman presents to the clinic with several concerns. "
    system_prompt += "Over the past several months, she has experienced thickening of her skin around her hands, forearms, and face.  "
    system_prompt += "She does not have any joint pain, stiffness, or family history of autoimmune diseases. "
    system_prompt += "She also does not ave any red or purple patches on her skin. "
    system_prompt += "She denies any rapid weight gain on her face, arms or legs. "
    system_prompt += "She denies taking any medications.'"
    
    return system_prompt

In [ ]:
def convert_to_vignette_prompt_gpt4(pat_dialogues):
    system_prompt = "Convert the following vignette into 3rd person."
    system_prompt += " It is a patient describing their medical symptoms."
    system_prompt += " Do not use the words 'AI language model' or references to the 'paragraph' mentioned in the vignette."
    system_prompt += " Do not create new information. - .\n"
    system_prompt += pat_dialogues
    system_prompt += "\n\nFor example: \nOriginal Vignette - 'I have a hard, yellowish-white horn-like growth"
    system_prompt += " on my head that started as a small, hard bump a few months ago and has grown bigger,"
    system_prompt += " and hurts when accidentally hit. I am a 60-year-old man."
    system_prompt += " I'm sorry, but I do not have that information as I am an AI language model." 
    system_prompt += " I am an AI language model, and based on the given paragraph, there is no information available regarding"
    system_prompt += " previous growths or history of skin cancer or other cancers. "
    system_prompt += " As an AI language model, it is not mentioned in the given paragraph whether"
    system_prompt += " I have tried any treatments or remedies for the horn-like growth."
    system_prompt += " The growth has a very firm texture, and it is located superficially on the skin,"
    system_prompt += " just beside the midline on the superior aspect of the skull."
    system_prompt += " As described in the given paragraph, the patient denies any pain at rest,"
    system_prompt += " but experiences pain when the lesion is accidentally struck;"
    system_prompt += " there is no mention of itch or bleeding. As an AI language model, there is no information available"
    system_prompt += " in the given paragraph regarding any comorbidities or previous skin disease diagnoses."
    system_prompt += " As mentioned in the given paragraph, the patient noticed a small,"
    system_prompt += " hard lesion a few months ago, which has grown progressively larger from that time to now."
    system_prompt += " However, there is no mention of any changes in color or shape over time.' "    
    system_prompt += "\n\nConverted Vignette - 'A 60-year-old man reports a hard, yellowish-white horn-like growth"
    system_prompt += " on his head that started as a small, hard bump a few months ago."
    system_prompt += " He says that it has grown bigger and hurts when accidently hit."
    system_prompt += " He says that there are no previous growths or history of skin cancer or other cancers."
    system_prompt += " He does not know if he has tried any treatments or remedies for the horn-like growth."
    system_prompt += " He says that the The growth has a very firm texture, and it is located superficially on the skin, just beside the midline on the superior aspect of the skull."
    system_prompt += " He denies any any pain at rest, but experiences pain when the lesion is accidentally struck."
    system_prompt += " There is no mention of itch or bleeding. He does not have information regarding any comorbidities or previous skin disease diagnoses."
    system_prompt += " There is no mention of any changes in color or shape over time.'"
    
    return system_prompt

In [ ]:
def convert_to_vignette_gpt3(pat_dialogues):
    convo = [{"role":"system", "content":convert_to_vignette_prompt_gpt3(pat_dialogues)}]
    response = call_gpt3_api(convo, n_responses=1, max_tokens=500)
    return response


def convert_to_vignette_gpt4(pat_dialogues):
    convo = [{"role":"system", "content":convert_to_vignette_prompt_gpt4(pat_dialogues)}]
    response = call_gpt3_api(convo, n_responses=1, max_tokens=500)
    return response

In [ ]:
def get_patient_responses(convo):
    temp = ""
    for c in convo:
        if c["role"]=="user":
            temp += c["content"]
            temp += " "
    return temp

In [ ]:
public_cases = ["public_case0"+str(x) for x in range(1,10)] + ["public_case"+str(x) for x in range(10,101)]
private_cases = ["private_case0"+str(x) for x in range(1,10)] + ["private_case"+str(x) for x in range(10,41)]
all_cases = public_cases + private_cases

In [ ]:
# GPT-3.5 conversations
res_df = pd.DataFrame(index=range(10), columns=range(len(all_cases)))

res = json.load(open("./results/conversations_raw/conversations_gpt3.json","r"))
for case in tqdm(all_cases):
    for j in range(10):
        vig = get_patient_responses(res[case][f"trial_{j}_doctor_responses_with_exam"])
        converted_vig = convert_to_vignette_gpt3(vig)
        res_df.loc[j,i] = converted_vig
        
res_df.to_csv("./results/conversational_summarization_gpt3.tsv", sep="\t")

In [ ]:
# GPT-4 conversations
res_df = pd.DataFrame(index=range(10), columns=range(len(all_cases)))

res = json.load(open("./results/conversations_raw/conversations_gpt4.json","r"))
for case in tqdm(all_cases):
    for j in range(10):
        vig = get_patient_responses(res[case][f"trial_{j}_doctor_responses_with_exam"])
        converted_vig = convert_to_vignette_gpt4(vig)
        res_df.loc[j,i] = converted_vig
        
res_df.to_csv("./results/conversational_summarization_gpt4.tsv", sep="\t")